In [23]:
import pandas as pd
import xlsxwriter

In [24]:
#list of relevant columns in each file
cols_E = ["Datetime","PI-401.PV","PI-402.PV","PI-403.PV","PIC-104.PV","PIC-105.PV"]
cols_H = ["Datetime","PI-351.PV","PI-301.PV","PI-302.PV","PI-303.PV","PI-304.PV","PI-107.PV","PI-108.PV"]

In [25]:
csv_files = ["H.csv","E.csv"]

#function to clean files and return relevant columns
def clean_file(file):
    df = pd.read_csv(file,header=4,encoding='utf-8')
    df = df.iloc[8:]
    df["Datetime"] = df["Unnamed: 2"] +" "+ df["Unnamed: 3"]
    if file == "H.csv":
        cols = cols_H
    elif file == "E.csv":
        cols = cols_E
    else: 
        print("Please rename files!")
    df = df[cols]
    
    for col in df.columns:
        if col != "Datetime":
            df[col] = pd.to_numeric(df[col])
        else:
            df[col] = pd.to_datetime(df[col])
    return(df)

In [26]:
#apply function to clean files to csv files
dictionary = {}
for file in csv_files:
    dictionary[file] = clean_file(file)

In [27]:
#merge dataframes into final df
final = pd.merge(dictionary["H.csv"],dictionary["E.csv"],on=["Datetime"])
final[["PIC-104.PV","PIC-105.PV"]] = (final[["PIC-104.PV","PIC-105.PV"]]/51.7149).round(1)

In [28]:
#extract month and year elements
month = final["Datetime"].iloc[0].month
year = final["Datetime"].iloc[0].year

In [29]:
#define setpoints for each PRD
setpoints = {"PI-351.PV":14.0,"PI-401.PV":15.0,"PI-402.PV":15.0,"PI-403.PV":15.0,"PI-301.PV":30.0,"PI-302.PV":30.0,"PI-303.PV":30.0,"PI-304.PV":20.0,"PIC-104.PV":25.0,"PIC-105.PV":25.0,"PI-108.PV":50.0,"PI-107.PV":55.0}

In [30]:
#create list of all PRDs
cols_total = cols_E[1:] + cols_H[1:]

In [31]:
#define function to filter data for values > setpoint
def filter_data(PRD):
    new = final[final[PRD] > setpoints[PRD]]
    new = new[["Datetime",PRD]]
    return(new)

In [32]:
#apply filter_data function
filtered_df = {}
for PRD in cols_total:
    filtered_df[PRD] = filter_data(PRD)

In [57]:
#setup workbook and worksheet
workbook = xlsxwriter.Workbook('PRD_Data_%d_%d.xlsx' %(year,month))
worksheet = workbook.add_worksheet("%d-%d" % (year,month))
bold = workbook.add_format({'bold':True})
worksheet.set_column('A:A',20)
worksheet.set_column('B:B',10)
worksheet.set_column('C:C',15)

#write filtered data into cells
row = 0
col = 0
for df in filtered_df:
    for element in filtered_df[df]:
        worksheet.write(row,col,element)
        col += 1
        worksheet.write(row,col,"Setpoint: %d psig" %setpoints[df])
    row += 1
    col = 0
    if filtered_df[df].empty == True:
        worksheet.write(row,col,"No exceedances above setpoint in %d-%d." % (year,month))
        row += 1
    else:
        for i,df_row in filtered_df[df].iterrows():
            date = df_row['Datetime'].strftime('%Y-%m-%d %H:%M')
            worksheet.write(row,col,date)
            col += 1
            worksheet.write(row,col,df_row[df])
            row += 1
            col = 0
worksheet.write(row,col,"Routine Sulfatreat changeouts exceedances are exempt; verify dates.", bold)
workbook.close()